# récupération des données du ficher "Variables La Serve.csv"
on ne reécupère que 2 colonnes :
  - 'Libellé', qui correspond 
    - au libellé du groupe (si la colonne 'Ad API' est vide)  
    - au libellé du groupe (si la colonne 'Ad API' contient une adresse) 
  - 'Ad API', qui contient l'adresse, et le type de variable
    

In [1]:
import csv
import re
from fileWriter import cls_TextFileWriter
verbose = True

In [2]:
fileCsvName  	= "./Nouvergies - La Serve/Variables La Serve.csv"
delimiter 		= ";"

In [3]:
csvData = {}
reqName = "xxx"
csvData[ reqName ] = {}

with open(fileCsvName) as csvfile:
	reader = csv.DictReader(csvfile, delimiter=delimiter)

	for row in reader:
		#print(row)
		if 'Libellé' in row.keys() :
			label 		= str(row['Libellé']).replace("     ","").replace(" ","_").replace(":","")
			addrAPI 	= row['Ad API']
			

		if addrAPI == "" :
			reqName = label
			csvData[ reqName ] = {}
		else :
			csvData[ reqName ][ label ] = addrAPI


# Affichage des groupes (requêtes) trouvés

In [4]:
for req in csvData:	
		print(" ---- '" + str(req) + "' ---- " )
		for k in csvData[req] :
			print("   " + str(k) + " --> " + str(csvData[req][k]) )


 ---- 'xxx' ---- 
   Défaut_mesure_de_niveau_amont_grille --> %M101
   Défaut_mesure_de_niveau_aval_centrale --> %M103
   Défaut_niveau_bas --> %M104
   Alarme_niveau_haut --> %M105
   Alarme_niveau_bas --> %M112
   Défaut_acquisition_température_centrale --> %M115
   Défaut_acquisition_température_extérieur --> %M116
   Protection_de_découplage --> %M117
   Protection_de_découplage_longue --> %M118
   Arrêt_d'urgence_général --> %M121
   Protection_C13-100 --> %M128
   Défaut_Com_entrées_sorties_déportées_Ilot_armoire_commun --> %M137
   Défaut_module_entrées_sorties_déportées_Ilot_armoire_commun --> %M138
   Interrupteur_HTA_général_ouvert --> %M139
   Défaut_dégrilleur --> %M147
   Retour_de_marche_dégrilleur --> %M148
   Défaut_manque_réactif_prolongé --> %M159
   Défaut_trop_de_réactif_prolongé --> %M160
 ---- 'ETAT_GROUPE' ---- 
   Groupe_en_Maintenance_ --> %M2001
   Groupe_en_Manu --> %M2002
   Groupe_en_Auto --> %M2003
   Commutateur_groupe_sur_arrêt --> %M2004
   Commutateur_

# défintion des constantes 

In [5]:

ReadFctCode 	= 4
WriteFctCode 	= 0
#StartReg 		= min(addrList)
#NbReg			= len(addrList)
EnableReading	= 1
EnableWritting	= 2
Option1			= 0
Option2			= 0

varIndex		= 1		# compteur
varReqIndex		= 1		# index du groupe (requête)
varName			= "?"
varType			= 8
varSigned		= 1
varPosition		= 1
varOption1		= ""
varOption2		= ""
varCoeffA		= 1
varCoeffB		= 0
varUnit			= "?"
varAction		= 2




## fonction de conversion des type de données

In [6]:
def dataTypeTranslate(wordType="MW") :
	d = {
		"MW"		: 8	, 		# /!\ TODO : à vérifier /!\
		"M" 		: 4	, 		# /!\ TODO : à vérifier /!\
	}

	try :
		varType = d[wordType]
	except :
		varType = str(wordType) + "???"

	return varType

# construction du fichier modbus  
  -  utilsation d'un regex '%(\w)(\d+)':
	- '%' ets le début du mot   
	- ([A-Z]+) est un chaine de lettres qui correspond au type de données    
	- ([0-9]+) est un chaine de chiffres qui correspond à l'adresse,  
	 ... on pourrait utiliser (\d+) ?  


In [7]:

modbusData = {}
#print(csvData)

reqIdx = 0
for req in csvData:
	varDict		= {}
	addrList 	= []

	# parcours de la liste des variables
	vCnt = 0
	for v in csvData[req] :
		#print(str(k) + " --> " + str(csvData[req][v]) )

		# Nom de la variable
		varDict[vCnt] = {}
		varDict[vCnt]['varName'] = v

		# récupération de l'adresse et du type de données
		#regex = '%([A-Z]+)(\d+)'
		regex = '%([A-Z]+)([0-9]+)'
		reAddr = re.match(regex, csvData[req][v])
		if reAddr :     # N° non trouvé ...
			wordType    = reAddr.groups()[0]    # W
			wordAddr    = reAddr.groups()[1]    # N° ligne
			addrList.append(wordAddr)
			print( str(wordType) + " - " + str(wordAddr) )

			varDict[vCnt]['varType'] 		= dataTypeTranslate(wordType)		# TODO : vérifier la conversion lettre -> chiffre ???
			varDict[vCnt]['varAddr'] 		= wordAddr							# TODO : vérifier l'adresse du registre (... est-ce qu'il faut multiplier par 2 ou 4 si on a plusieurs octets dans le mot ?)
			#varDict[vCnt]['varPosition'] = varDict[vCnt]['varAddr'] - StartReg

			vCnt += 1

	if len(addrList) : 		# si la liste n'est pas vide
		#StartReg 	= addrList[0]		# TODO vérifier que c'est bien le min, et non la 1ère valeur <addrList[0]> qu'il faut 
		StartReg 	= min(addrList)		# TODO vérifier que c'est bien le min, et non la 1ère valeur <addrList[0]> qu'il faut 
		NbReg		= len(addrList)

	# table de description des requêtes (Modbus_RequestsTables)
	if len(addrList) : 		# si la liste n'est pas vide
		reqDef=	str(reqIdx) 		+ ";" \
			+ 	str(req) 			+ ";" \
			+ 	str(ReadFctCode)  	+ ";" \
			+ 	str(ReadFctCode) 	+ ";" \
			+ 	str(StartReg) 		+ ";" \
			+ 	str(NbReg) 			+ ";" \
			+ 	str(EnableReading) 	+ ";" \
			+ 	str(EnableWritting) + ";" \
			+ 	str(Option1) 		+ ";" \
			+ 	str(Option2) 		
		if verbose :
			print(reqDef) 

		modbusData[req] = {}
		modbusData[req]['id'] 		= reqIdx
		modbusData[req]['Request'] 	= reqDef
		modbusData[req]['varDict'] 	= varDict
		
		if 0 : 
			print(addrList)
			print(	str(len(csvData[req])) + " --> " + str(csvData[req]) )
			print()
	#else : 	print( "/!\\ groupe vide : " + str(req) + " --> " + str(csvData[req]) )

	reqIdx += 1



M - 101
M - 103
M - 104
M - 105
M - 112
M - 115
M - 116
M - 117
M - 118
M - 121
M - 128
M - 137
M - 138
M - 139
M - 147
M - 148
M - 159
M - 160
0;xxx;4;4;101;18;1;2;0;0
M - 2001
M - 2002
M - 2003
M - 2004
M - 2005
M - 2003
M - 2005
M - 2006
M - 2006
M - 2009
M - 2010
M - 2009
M - 2010
M - 2017
M - 2018
M - 2019
M - 2020
M - 2021
M - 2022
M - 2023
M - 2024
M - 2025
M - 2026
M - 2027
M - 2028
M - 2029
M - 2030
M - 2032
M - 2033
M - 2034
M - 2035
M - 2036
M - 2037
M - 2038
M - 2040
M - 2044
M - 2045
M - 2046
M - 2047
M - 2048
M - 2049
M - 2050
M - 2051
M - 2053
M - 2054
M - 2055
M - 2056
M - 2057
M - 2058
M - 2059
M - 2060
M - 2061
M - 2062
M - 2063
M - 2064
M - 2065
M - 2066
M - 2073
M - 2080
M - 2081
M - 2082
M - 2083
M - 2084
M - 2091
M - 2092
M - 2093
M - 2095
M - 2096
M - 2097
M - 2098
M - 2099
M - 2100
M - 2113
M - 2114
M - 2115
M - 2116
M - 2122
M - 2123
M - 2125
M - 2126
M - 2127
M - 2129
M - 2130
M - 2131
M - 2132
M - 2133
M - 2134
M - 2135
M - 2136
M - 2137
M - 2138
M - 2139
M -

# affichage des données utiles au fichier modbus

In [8]:
if verbose :
	# parcours des requêtes
	for req in modbusData:					
		print(" ---- " + str(req) + " ---- ")
		for k in modbusData[req] :
			print("      " + str(k) + " ---> " + str(modbusData[req][k]) )
		print()

 ---- xxx ---- 
      id ---> 0
      Request ---> 0;xxx;4;4;101;18;1;2;0;0
      varDict ---> {0: {'varName': 'Défaut_mesure_de_niveau_amont_grille', 'varType': 4, 'varAddr': '101'}, 1: {'varName': 'Défaut_mesure_de_niveau_aval_centrale', 'varType': 4, 'varAddr': '103'}, 2: {'varName': 'Défaut_niveau_bas', 'varType': 4, 'varAddr': '104'}, 3: {'varName': 'Alarme_niveau_haut', 'varType': 4, 'varAddr': '105'}, 4: {'varName': 'Alarme_niveau_bas', 'varType': 4, 'varAddr': '112'}, 5: {'varName': 'Défaut_acquisition_température_centrale', 'varType': 4, 'varAddr': '115'}, 6: {'varName': 'Défaut_acquisition_température_extérieur', 'varType': 4, 'varAddr': '116'}, 7: {'varName': 'Protection_de_découplage', 'varType': 4, 'varAddr': '117'}, 8: {'varName': 'Protection_de_découplage_longue', 'varType': 4, 'varAddr': '118'}, 9: {'varName': "Arrêt_d'urgence_général", 'varType': 4, 'varAddr': '121'}, 10: {'varName': 'Protection_C13-100', 'varType': 4, 'varAddr': '128'}, 11: {'varName': 'Défaut_Com_ent

# Fichier Modbus

In [9]:
lines = []
lines.append(
"""#Based on La Serve collected values

# Description of fields
# Id;Name;ReadFctCode;WriteFctCode;StartReg;NbReg;EnableReading;EnableWritting;Option1;Option2""")

#### 	Modbus_RequestsTables :
lines.append("Modbus_RequestsTables={")
for req in modbusData:
	if 'Request' in modbusData[req] :
		lines.append(modbusData[req]['Request'] )
lines.append("}")
lines.append("")
lines.append("")

lines.append(
"""# Description of fields
# varIndex;varReqIndex;varName;varType;varSigned;varPosition;varOption1;varOption2;varCoeffA;varCoeffB;varUnit;varAction
# """)

####	Modbus_VariablesTables :
lines.append("Modbus_VariablesTables={")
for req in modbusData:
	if 'variable' in modbusData[req] :
		lines.append(modbusData[req]['variable'] )
lines.append( "}")


#########
if verbose : 
	for l in lines :
		print(l)


#Based on La Serve collected values

# Description of fields
# Id;Name;ReadFctCode;WriteFctCode;StartReg;NbReg;EnableReading;EnableWritting;Option1;Option2
Modbus_RequestsTables={
0;xxx;4;4;101;18;1;2;0;0
1;ETAT_GROUPE;4;4;2001;106;1;2;0;0
2;NIVEAUX;4;4;110;11;1;2;0;0
3;DEBITS;4;4;126;3;1;2;0;0
4;TEMPERATURES;4;4;2030;4;1;2;0;0
5;INFOS_GROUPE;4;4;2000;6;1;2;0;0
6;MESURES_ELECTRIQUES;4;4;135;13;1;2;0;0
7;GROUPE_1;4;4;1108;9;1;2;0;0
8;GROUPE_2;4;4;2108;9;1;2;0;0
9;GHY_ET_GRAISSAGE;4;4;2003;4;1;2;0;0
10;GHY_ET_GRAISSAGE_GROUPE_1;4;4;1005;6;1;2;0;0
11;GHY_ET_GRAISSAGE_GROUPE_2;4;4;2005;6;1;2;0;0
12;TEMPERATURES_G1;4;4;1201;4;1;2;0;0
13;TEMPERATURES_G2;4;4;2201;4;1;2;0;0
14;GROUPES;4;4;150;1;1;2;0;0
15;REGULATION_DE_NIVEAU;4;4;190;6;1;2;0;0
16;REGLAGES;4;4;1101;6;1;2;0;0
17;HEURE;4;4;100;6;1;2;0;0
18;COMMUNICATION;4;4;1020;10;1;2;0;0
19;REGLAGES_DEGRILLEUR_GROUPE_1;4;4;4000;2;1;2;0;0
20;_DEGRILLEUR_GROUPE_1;4;4;4007;8;1;2;0;0
21;REGLAGES_DEGRILLEUR_GROUPE_2;4;4;4050;2;1;2;0;0
22;_DEGRILLEUR

In [10]:
# écriture du fichier text
out= cls_TextFileWriter(outputDir= "./out/Nuovergies/")		#définiton du répertoire
out.wr(txt=lines, fileName= "modbus_La_Serve.ini")			#ecriture des données de la list de lignes


>>> création du répertoire './out/Nuovergies/'

 --- fichier ./out/Nuovergies/modbus_La_Serve.ini créé ---- 
